In [ ]:
import sqlite3
import pandas as pd

def fetch_balanced_data(db_name, epochs):
    conn = sqlite3.connect(db_name)
    c = conn.cursor()
    
    # Placeholder lists for data
    data = []
    
    for epoch_i in range(1, epochs+1):
        # Fetch an equal number of human and machine translations for the epoch
        query = """
        SELECT epoch_i_list, src_sentences_converted_logging_org, tgt_sentences_converted_logging_org, '1' AS ht_or_mt_target
        FROM translations_valid
        WHERE epoch_i_list=?
        UNION ALL
        SELECT epoch_i_list, src_sentences_converted_logging_org, fake_tgt_sentences_converted_logging_G2_train, '0'
        FROM translations_valid
        WHERE epoch_i_list=?
        UNION ALL
        SELECT epoch_i_list, src_sentences_converted_logging_org, fake_tgt_sentences_G1_pretrain_converted_logging, '0'
        FROM translations_valid
        WHERE epoch_i_list=?
        LIMIT ?
        """
        # Assuming you want N examples from each category (human, G2, G1) per epoch
        N = 100  # Adjust N based on your data distribution and desired dataset size
        c.execute(query, (epoch_i, epoch_i, epoch_i, N*3))
        data.extend(c.fetchall())
    
    conn.close()
    
    # Create a DataFrame
    df = pd.DataFrame(data, columns=['epoch', 'src_sentence', 'translation', 'ht_or_mt_target'])
    return df


: 